# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPY/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,piti village,13.4626,144.6933,25.03,85,20,2.06,GU,1709413832
1,1,outjo,-20.1167,16.1500,25.13,37,74,2.99,NaN,1709414004
2,2,changji,44.0167,87.3167,-9.56,79,100,1.00,CN,1709414004
3,3,kencong,-8.2833,113.3667,25.06,90,100,1.12,ID,1709414004
4,4,longyearbyen,78.2186,15.6401,-8.09,73,75,1.03,SJ,1709413725


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates_df = city_data_df[["City", "Lat", "Lng", "Humidity"]]
map_plot_1 = coordinates_df.hvplot.points(
    "Lng", "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    #xlabel="X", # personally I prefer to have the labels as Latitude and Longitude
    #ylabel="Y"
)
    
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
# my ideal weather conditions
# Max temp <25 , >=15
# Humidity < 80%
# Wind speed < 5 m/s
# Cloudiness < 50

ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 25) &
                                (city_data_df["Max Temp"] >= 15) & 
                                (city_data_df["Humidity"] < 80) & 
                                (city_data_df["Wind Speed"] < 5) &
                                (city_data_df["Cloudiness"] < 50)]



# Drop any rows with null values
ideal_weather_cleaned_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_cleaned_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,ascension,30.2169,-90.9001,23.12,77,40,2.06,US,1709414006
21,21,tura,25.5198,90.2201,17.83,45,0,1.08,IN,1709413928
36,36,khagaria,25.5000,86.4833,19.42,40,12,3.94,IN,1709414008
78,78,chonchi,-42.6128,-73.8092,15.62,48,16,0.91,CL,1709414015
108,108,holualoa,19.6228,-155.9522,24.91,70,0,4.12,US,1709414019
135,135,mildura,-34.2000,142.1500,15.99,43,0,4.51,AU,1709414022
148,148,adrar,20.5022,-10.0711,22.64,11,0,3.82,MR,1709414024
157,157,awjilah,29.1081,21.2869,16.67,32,47,3.98,LY,1709414025
201,201,arkadelphia,34.1209,-93.0538,22.87,53,0,4.12,US,1709414023
227,227,kallupatti,9.7167,77.8667,24.27,66,22,1.33,IN,1709414035


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cleaned_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,ascension,US,30.2169,-90.9001,77,
21,tura,IN,25.5198,90.2201,45,
36,khagaria,IN,25.5000,86.4833,40,
78,chonchi,CL,-42.6128,-73.8092,48,
108,holualoa,US,19.6228,-155.9522,70,
135,mildura,AU,-34.2000,142.1500,43,
148,adrar,MR,20.5022,-10.0711,11,
157,awjilah,LY,29.1081,21.2869,32,
201,arkadelphia,US,34.1209,-93.0538,53,
227,kallupatti,IN,9.7167,77.8667,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000


params = {
    "categories": "accommodation.hotel",
    #"apiKey": geoapify_key
       
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["apiKey"] = geoapify_key
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(name_address.url)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ascension - nearest hotel: Cajun Country Inn
tura - nearest hotel: No hotel found
khagaria - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
holualoa - nearest hotel: Kona Hotel
mildura - nearest hotel: Chaffey Motor Inn
adrar - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
arkadelphia - nearest hotel: Captain Henderson House
kallupatti - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
rahuri - nearest hotel: No hotel found
udon thani - nearest hotel: UDtel
san narciso - nearest hotel: Kapitan Liwa
salalah - nearest hotel: Muscat International Hotel
alaghsas - nearest hotel: Hôtel Zagado
siwa oasis - nearest hotel: فندق الكيلانى
coquimbo - nearest hotel: Hotel Iberia
saint-pierre - nearest hotel: Tropic Hotel
francistown - nearest hotel: City Inn
timimoun - nearest hotel: أناف
praia da vitoria - nearest hotel: Salles
taoudenni - nearest hotel: No hotel found
jinghong - nearest hotel: โรงแรมสืเบา
t

,City,Country,Lat,Lng,Humidity,Hotel Name
17,ascension,US,30.2169,-90.9001,77,Cajun Country Inn
21,tura,IN,25.5198,90.2201,45,No hotel found
36,khagaria,IN,25.5000,86.4833,40,No hotel found
78,chonchi,CL,-42.6128,-73.8092,48,Hotel Huildín
108,holualoa,US,19.6228,-155.9522,70,Kona Hotel
135,mildura,AU,-34.2000,142.1500,43,Chaffey Motor Inn
148,adrar,MR,20.5022,-10.0711,11,No hotel found
157,awjilah,LY,29.1081,21.2869,32,No hotel found
201,arkadelphia,US,34.1209,-93.0538,53,Captain Henderson House
227,kallupatti,IN,9.7167,77.8667,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    hover_cols = ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)
    

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)